Load the Data in Azure Databricks: Use the CSV data stored in ADLS.

In [0]:
file_path = "/mnt/dataset/loan.csv"
df = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load(file_path)
df.show()

+-----------+---+------+-------------------+--------------+-----------+------+-----------+-------------+------------------+-----------+-------+------------+----------------+------------------+
|Customer_ID|Age|Gender|         Occupation|Marital Status|Family Size|Income|Expenditure|Use Frequency|     Loan Category|Loan Amount|Overdue| Debt Record| Returned Cheque| Dishonour of Bill|
+-----------+---+------+-------------------+--------------+-----------+------+-----------+-------------+------------------+-----------+-------+------------+----------------+------------------+
|    IB14001| 30|  MALE|       BANK MANAGER|        SINGLE|          4| 50000|      22199|            6|           HOUSING| 10,00,000 |      5|      42,898|               6|                 9|
|    IB14008| 44|  MALE|          PROFESSOR|       MARRIED|          6| 51000|      19999|            4|          SHOPPING|     50,000|      3|      33,999|               1|                 5|
|    IB14012| 30|FEMALE|           

Clean the Data:

Remove leading/trailing spaces from column names.
Handle any null or inconsistent values.

In [0]:
from pyspark.sql.functions import col

df = df.select([col(c).alias(c.strip()) for c in df.columns])
df.show()

+-----------+---+------+-------------------+--------------+-----------+------+-----------+-------------+------------------+-----------+-------+-----------+---------------+-----------------+
|Customer_ID|Age|Gender|         Occupation|Marital Status|Family Size|Income|Expenditure|Use Frequency|     Loan Category|Loan Amount|Overdue|Debt Record|Returned Cheque|Dishonour of Bill|
+-----------+---+------+-------------------+--------------+-----------+------+-----------+-------------+------------------+-----------+-------+-----------+---------------+-----------------+
|    IB14001| 30|  MALE|       BANK MANAGER|        SINGLE|          4| 50000|      22199|            6|           HOUSING| 10,00,000 |      5|     42,898|              6|                9|
|    IB14008| 44|  MALE|          PROFESSOR|       MARRIED|          6| 51000|      19999|            4|          SHOPPING|     50,000|      3|     33,999|              1|                5|
|    IB14012| 30|FEMALE|            DENTIST|      

In [0]:
df.describe().show()

+-------+-----------+-----------------+------+---------------+--------------+----------------+-----------------+------------------+------------------+-------------+-----------+------------------+-----------------+-----------------+------------------+
|summary|Customer_ID|              Age|Gender|     Occupation|Marital Status|     Family Size|           Income|       Expenditure|     Use Frequency|Loan Category|Loan Amount|           Overdue|      Debt Record|  Returned Cheque| Dishonour of Bill|
+-------+-----------+-----------------+------+---------------+--------------+----------------+-----------------+------------------+------------------+-------------+-----------+------------------+-----------------+-----------------+------------------+
|  count|        500|              500|   500|            500|           500|             500|              468|               481|               500|          500|        500|               500|              500|              500|               5

### Income Distribution
### Analyze the range and distribution of customer income. You can also categorize customers into income brackets.

In [0]:
from pyspark.sql.functions import when

income_bracket_df = df.withColumn(
    "Income_Bracket",
    when(df["Income"] < 30000, "Low Income")
    .when((df["Income"] >= 30000) & (df["Income"] <= 60000), "Middle Income")
    .otherwise("High Income")
)

income_bracket_df.groupBy("Income_Bracket").count().show()

+--------------+-----+
|Income_Bracket|count|
+--------------+-----+
| Middle Income|  268|
|   High Income|  230|
|    Low Income|    2|
+--------------+-----+



In [0]:
display(income_bracket_df.groupBy("Income_Bracket").count())

Income_Bracket,count
Middle Income,268
High Income,230
Low Income,2


Databricks visualization. Run in Databricks to view.

### Loan Categories and Frequency
### Find the most common loan categories and their frequencies.

In [0]:
loan_category_df = df.groupBy("Loan Category").count().orderBy("count", ascending=False)
loan_category_df.show()

+------------------+-----+
|     Loan Category|count|
+------------------+-----+
|         GOLD LOAN|   77|
|           HOUSING|   67|
|        AUTOMOBILE|   60|
|        TRAVELLING|   53|
|       RESTAURANTS|   41|
|COMPUTER SOFTWARES|   35|
|          SHOPPING|   35|
|          BUSINESS|   24|
|  EDUCATIONAL LOAN|   20|
|        RESTAURANT|   20|
|           DINNING|   14|
|       ELECTRONICS|   14|
|   HOME APPLIANCES|   14|
|       AGRICULTURE|   12|
|       BOOK STORES|    7|
|          BUILDING|    7|
+------------------+-----+



In [0]:
display(loan_category_df)

Loan Category,count
GOLD LOAN,77
HOUSING,67
AUTOMOBILE,60
TRAVELLING,53
RESTAURANTS,41
COMPUTER SOFTWARES,35
SHOPPING,35
BUSINESS,24
EDUCATIONAL LOAN,20
RESTAURANT,20


Databricks visualization. Run in Databricks to view.

### Overdue Analysis
### Identify customers with high overdue amounts and compare it with their income and expenditure.

In [0]:
from pyspark.sql.functions import expr

overdue_analysis_df = df.withColumn(
    "Overdue_to_Income", expr("Overdue / Income")
).withColumn(
    "Overdue_to_Expenditure", expr("Overdue / Expenditure")
)
overdue_analysis_df.orderBy("Overdue_to_Income", ascending=False).show()

+-----------+---+------+-----------------+--------------+-----------+------+-----------+-------------+------------------+-----------+-------+-----------+---------------+-----------------+--------------------+----------------------+
|Customer_ID|Age|Gender|       Occupation|Marital Status|Family Size|Income|Expenditure|Use Frequency|     Loan Category|Loan Amount|Overdue|Debt Record|Returned Cheque|Dishonour of Bill|   Overdue_to_Income|Overdue_to_Expenditure|
+-----------+---+------+-----------------+--------------+-----------+------+-----------+-------------+------------------+-----------+-------+-----------+---------------+-----------------+--------------------+----------------------+
|    IB14693| 27|  MALE|           DOCTOR|        SINGLE|          7| 28366|      29258|            2|         GOLD LOAN|  1,659,986|      9|      20611|              5|                5| 3.17281252203342E-4|  3.076081755417321...|
|    IB14685| 26|  MALE|     BANK MANAGER|        SINGLE|          5| 29

In [0]:
display(overdue_analysis_df.orderBy("Overdue_to_Income", ascending=False))

Customer_ID,Age,Gender,Occupation,Marital Status,Family Size,Income,Expenditure,Use Frequency,Loan Category,Loan Amount,Overdue,Debt Record,Returned Cheque,Dishonour of Bill,Overdue_to_Income,Overdue_to_Expenditure
IB14693,27,MALE,DOCTOR,SINGLE,7,28366,29258,2,GOLD LOAN,"1,659,986",9,20611,5,5,3.17281252203342E-4,3.0760817554173215E-4
IB14685,26,MALE,BANK MANAGER,SINGLE,5,29565,19490,9,HOUSING,"1,767,908",9,72653,4,6,3.0441400304414006E-4,4.617752693689071E-4
IB14619,43,MALE,TEACHER,SINGLE,4,35020,26286,8,SHOPPING,"162,449",9,22862,3,3,2.569960022844089E-4,3.423875827436658E-4
IB14459,32,MALE,DOCTOR,MARRIED,4,35472,18340,6,SHOPPING,"571,540",9,52731,0,0,2.537212449255751E-4,4.907306434023992E-4
IB14321,28,MALE,TEACHER,MARRIED,7,31747,17995,8,COMPUTER SOFTWARES,"720,712",8,66706,1,7,2.5199231423441587E-4,4.4456793553764934E-4
IB14380,41,FEMALE,NURSE,MARRIED,5,36784,null,4,HOUSING,"1,232,534",9,60243,3,1,2.446715963462375E-4,null
IB14070,40,MALE,PUBLIC WORKS,MARRIED,4,38000,20000,3,GOLD LOAN,"4,00,000",9,"19,954",3,2,2.3684210526315788E-4,4.5E-4
IB14210,40,MALE,PUBLIC WORKS,MARRIED,4,38000,20000,3,GOLD LOAN,"4,00,000",9,"19,954",3,2,2.3684210526315788E-4,4.5E-4
IB14285,40,MALE,PUBLIC WORKS,MARRIED,4,38000,20000,3,GOLD LOAN,"4,00,000",9,"19,954",3,2,2.3684210526315788E-4,4.5E-4
IB14086,51,FEMALE,TECHNICIAN,MARRIED,5,30000,null,5,RESTAURANTS,"1,25,463",7,"52,634",4,10,2.3333333333333333E-4,null


Databricks visualization. Run in Databricks to view.

### Expenditure Patterns
### Analyze expenditure patterns across family sizes or marital statuses.

In [0]:
expenditure_by_family_size = df.groupBy("Family Size").avg("Expenditure").orderBy("Family Size")
expenditure_by_family_size.show()

+-----------+------------------+
|Family Size|  avg(Expenditure)|
+-----------+------------------+
|          2| 30383.32786885246|
|          3|25790.014705882353|
|          4|25633.577981651375|
|          5|25842.114583333332|
|          6| 30232.19540229885|
|          7|           28854.2|
+-----------+------------------+



In [0]:
display(expenditure_by_family_size)

Family Size,avg(Expenditure)
2,30383.32786885246
3,25790.014705882353
4,25633.577981651375
5,25842.114583333332
6,30232.19540229885
7,28854.2


Databricks visualization. Run in Databricks to view.

### Debt and Payment Issues
### Identify customers with high counts of "Returned Cheque" or "Dishonour of Bill."

In [0]:
payment_issues_df = df.filter((df["Returned Cheque"] > 3) | (df["Dishonour of Bill"] > 3))
payment_issues_df.show()

+-----------+---+------+-------------------+--------------+-----------+------+-----------+-------------+------------------+-----------+-------+-----------+---------------+-----------------+
|Customer_ID|Age|Gender|         Occupation|Marital Status|Family Size|Income|Expenditure|Use Frequency|     Loan Category|Loan Amount|Overdue|Debt Record|Returned Cheque|Dishonour of Bill|
+-----------+---+------+-------------------+--------------+-----------+------+-----------+-------------+------------------+-----------+-------+-----------+---------------+-----------------+
|    IB14001| 30|  MALE|       BANK MANAGER|        SINGLE|          4| 50000|      22199|            6|           HOUSING| 10,00,000 |      5|     42,898|              6|                9|
|    IB14008| 44|  MALE|          PROFESSOR|       MARRIED|          6| 51000|      19999|            4|          SHOPPING|     50,000|      3|     33,999|              1|                5|
|    IB14018| 29|  MALE|            TEACHER|      

In [0]:
display(payment_issues_df)

Customer_ID,Age,Gender,Occupation,Marital Status,Family Size,Income,Expenditure,Use Frequency,Loan Category,Loan Amount,Overdue,Debt Record,Returned Cheque,Dishonour of Bill
IB14001,30,MALE,BANK MANAGER,SINGLE,4,50000,22199,6,HOUSING,"10,00,000",5,"42,898",6,9
IB14008,44,MALE,PROFESSOR,MARRIED,6,51000,19999,4,SHOPPING,"50,000",3,"33,999",1,5
IB14018,29,MALE,TEACHER,MARRIED,5,45767,12787,3,GOLD LOAN,"6,00,000",7,"11,000",0,4
IB14025,39,FEMALE,TEACHER,MARRIED,6,46619,18675,4,HOUSING,"12,09,867",8,"29,999",6,8
IB14027,51,MALE,SYSTEM MANAGER,MARRIED,3,49999,19111,5,RESTAURANTS,"60,676",8,"13,000",2,5
IB14029,24,FEMALE,TEACHER,SINGLE,3,45008,17454,4,AUTOMOBILE,"3,99,435",9,"51,987",4,7
IB14031,37,FEMALE,SOFTWARE ENGINEER,MARRIED,5,55999,23999,5,AUTOMOBILE,"60,999",2,0,5,3
IB14034,32,MALE,PRODUCT ENGINEER,MARRIED,6,null,29000,7,COMPUTER SOFTWARES,"80,660",6,"4,500",5,4
IB14037,54,FEMALE,TEACHER,MARRIED,5,48099,19999,4,RESTAURANTS,"30,999",1,"12,000",7,5
IB14039,45,MALE,ACCOUNT MANAGER,MARRIED,7,45777,18452,4,GOLD LOAN,"9,87,611",7,"39,999",8,1


# Transform Data for Further Analysis

### Add Derived Columns
### Add meaningful columns such as total liabilities (sum of loan amount, overdue, and debt record).

In [0]:
from pyspark.sql.functions import col

df = df.withColumn(
    "Total_Liabilities",
    col("Loan Amount") + col("Overdue") + col("Debt Record")
)
df.select("Customer_ID", "Total_Liabilities").show()

+-----------+-----------------+
|Customer_ID|Total_Liabilities|
+-----------+-----------------+
|    IB14001|             NULL|
|    IB14008|             NULL|
|    IB14012|             NULL|
|    IB14018|             NULL|
|    IB14022|             NULL|
|    IB14024|             NULL|
|    IB14025|             NULL|
|    IB14027|             NULL|
|    IB14029|             NULL|
|    IB14031|             NULL|
|    IB14032|             NULL|
|    IB14034|             NULL|
|    IB14037|             NULL|
|    IB14039|             NULL|
|    IB14041|             NULL|
|    IB14042|             NULL|
|    IB14045|             NULL|
|    IB14049|             NULL|
|    IB14050|             NULL|
|    IB14054|             NULL|
+-----------+-----------------+
only showing top 20 rows



### Customer Segmentation
### Segment customers based on usage frequency (low, medium, high).

In [0]:
df = df.withColumn(
    "Use_Frequency_Segment",
    when(df["Use Frequency"] <= 3, "Low")
    .when((df["Use Frequency"] > 3) & (df["Use Frequency"] <= 6), "Medium")
    .otherwise("High")
)

df.groupBy("Use_Frequency_Segment").count().show()

+---------------------+-----+
|Use_Frequency_Segment|count|
+---------------------+-----+
|                 High|  147|
|                  Low|  104|
|               Medium|  249|
+---------------------+-----+



# Save Processed Data for Reporting


In [0]:
df.write.format("parquet").mode("overwrite").save("/mnt/project5output", header="true", inferschema="true")

saving as CSV

In [0]:
df.write.format("csv").mode("overwrite").save("/mnt/project5output", header="true", inferschema="true")